<a href="https://colab.research.google.com/github/Dvaalmeyda/deteksi-kemiripan-judul-skripsi/blob/main/Deployment_Akhir_Skripsi_Diva_Putra_Almeyda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deployment - Gradio

### Dependencies

In [ ]:
!pip install transformers==4.36.0
!pip install keras==2.15.0
!pip install -U sentence-transformers
!pip install gradio
!pip install Sastrawi
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found e

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [ ]:
import gradio as gr
print(gr.__version__)

5.31.0


In [ ]:
# Gradio
import gradio as gr
# Data Preprocess
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#Predict
import numpy as np
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Download NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load stopwords Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/TERAKHIR-ADAM'

In [ ]:
# Load tokenizer model
indobertTokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-large-p2")

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
# Load Model
indobertModel = TFBertForSequenceClassification.from_pretrained(model_path, num_labels=2)

Some layers from the model checkpoint at /content/drive/MyDrive/TERAKHIR-ADAM were not used when initializing TFBertForSequenceClassification: ['dropout_341']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/TERAKHIR-ADAM.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


#### Compile Model

In [ ]:
# Optimizer and loss function
optimizerFinal = keras.optimizers.Adam(learning_rate=3e-5)
lossFinal = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Compile model
indobertModel.compile(
    optimizer=optimizerFinal,
    loss=lossFinal,
    metrics=[
        "accuracy"
    ]
)

### Load Dataset

In [ ]:
# URL dataset
skripsi_dataset_url = 'https://docs.google.com/spreadsheets/d/1YAuYZa_LFqxqOpiMgFr-imZRoiIaNSZGDscOvY30tb8/export?pli=1&gid=1020168489&format=csv'

In [ ]:
savedTitles_df = pd.read_csv(skripsi_dataset_url, usecols=['judul lama'])
# Remove Duplicates
savedTitles_df = savedTitles_df.drop_duplicates(subset=['judul lama'])
savedTitles_df

,judul lama
0,implementasi ic tl494 pada catu daya arus sear...
1,pemeliharaan jaringan distribusi tenaga listri...
2,perencanaan dan realisasi pembangkitan daya di...
3,sistem jaring distribusi tenaga listrik pt. pl...
4,pengujian pembangkit listrik tenaga angin deng...
...,...
142,transformator pada pt. pln pembangkitan jawa -...
144,"pembuatan simulasi rangkian resistor, indikato..."
146,perancangan home-page universitas tidar magela...
147,perancangan tapis aktif lolos piorde dua siste...


### Helper Function

#### Preprocess

In [ ]:
# Helper Function - Filtering & Casefolding
def remove_punctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.lower()
  return text

In [ ]:
# Helper Function Tokenisasi nltk
def tokenize_nltk_stopwords(text):
    # Tokenisasi
    tokens = word_tokenize(text)
    # Stopwords removal
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# Fungsi untuk menggabungkan token kembali menjadi kalimat
def join_tokens(tokens):
    return ' '.join(tokens)

In [ ]:
# Sub-main Function - Preprocessing
def preprocess_text(text):
    text = remove_punctuation(text)  # Filtering
    tokens = tokenize_nltk_stopwords(text)  # Tokenize & Stopwords Removal
    processed_text = join_tokens(tokens)
    return processed_text

In [ ]:
# Preprocessing judul untuk efisiensi waktu
saved_titles = savedTitles_df['judul lama'].apply(preprocess_text).tolist()

#### Predict

In [ ]:
# Helper Function - Input process & Predict
def preprocess_and_predict(input_title, saved_title):
    # Embeddings
    inputs = indobertTokenizer(
        input_title,
        saved_title,
        max_length=70,
        padding="max_length",
        truncation=True,
        return_tensors="tf",
    )
    # Predict
    predicted_raw = indobertModel.predict(
        [inputs["input_ids"], inputs["attention_mask"]], verbose=0
    )
    y_pred = np.argmax(predicted_raw["logits"], axis=1)
    label = y_pred[0]
    return  label #"Terindikasi Mirip" if label == 1 else "Terindikasi Tidak Mirip"

In [ ]:
def compare_titles(input_titles):
    results = []

    # Preprocess input titles only once
    preprocessed_input_titles = [preprocess_text(input_title) for input_title in input_titles]

    # Compare preprocessed input titles with already preprocessed saved titles
    for input_title, preprocessed_input_title in zip(input_titles, preprocessed_input_titles):
        for saved_title in saved_titles:
            label = preprocess_and_predict(preprocessed_input_title, saved_title)
            # Tambah keterangan
            keterangan = "Terindikasi Mirip" if label == 1 else "Terindikasi Tidak Mirip"
            results.append([input_title, saved_title, label, keterangan])

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["Judul Masukkan", "Judul Terarsip", "Label", "Keterangan"])
    return df

## Gradio

In [ ]:
import time

In [ ]:
# Main Function
def process_titles_and_time(input_titles):
  start_time = time.time()  # Timer start
  results_df = compare_titles(input_titles.split(", "))
  processing_time = time.time() - start_time  # Timer count
  return results_df, f"{processing_time:.2f} detik"

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Container utama dengan padding
    with gr.Column(scale=1, elem_classes="main-container"):
        # Header section
        with gr.Row(elem_classes="header-section"):
            gr.Markdown(
                """
                # Sistem Deteksi Kemiripan Judul Skripsi
                """
            )

        # Input section
        with gr.Column(elem_classes="input-section"):
            # Input area
            input_titles = gr.Textbox(
                lines=3,
                placeholder="Masukkan judul skripsi (pisahkan dengan koma jika lebih dari satu)",
                label="Masukkan Judul Skripsi",
                elem_classes="input-box"
            )
            with gr.Row(elem_classes="button-row"):
                process_button = gr.Button("Proses", elem_classes="custom-button")

         # Output section
        with gr.Column(elem_classes="output-section"):
            gr.Markdown("### Hasil Perbandingan")
            output_table = gr.Dataframe(
                headers=["Judul Masukan", "Judul Tersimpan", "Label", "Keterangan"],
                interactive=False,row_count=10,
                col_count=(4, "fixed"),
                wrap=True,
                type="pandas",  # Pastikan menggunakan tipe DataFrame
                elem_classes="result-table"
            )

            processing_time_text = gr.Textbox(
                label="Waktu Pemrosesan (detik)",
                interactive=False,
                elem_classes="processing-time"
            )
     # Soft pastel CSS
    gr.HTML("""
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            background-color: #f2f5f9 !important;
        }

        .gradio-container {
            max-width: 95% !important;
            margin: 0 auto !important;
            padding: 20px !important;
            background-color: #f2f5f9 !important;
        }

        .header-section {
            background-color: #ffffff !important;
            padding: 20px 30px !important;
            border-radius: 12px !important;
            margin-bottom: 20px !important;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05) !important;
        }

        .header-section h1 {
            color: #2b3a55 !important;
            font-size: 26px !important;
            margin: 0 !important;
            text-align: center !important;
        }

        .input-section {
            background-color: #ffffff !important;
            padding: 20px !important;
            border-radius: 12px !important;
            margin-bottom: 20px !important;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05) !important;
        }

        .input-box textarea {
            border: 1px solid #dce3ea !important;
            border-radius: 8px !important;
            padding: 12px !important;
            font-size: 15px !important;
            background-color: #f9fbfd !important;
            color: #212121 !important;
        }

        .button-row {
            justify-content: flex-end !important;
            padding: 10px 0 !important;
        }

        .custom-button {
            background-color: #6c9bcf !important;
            color: white !important;
            border-radius: 8px !important;
            padding: 10px 30px !important;
            font-size: 16px !important;
            cursor: pointer !important;
            transition: background-color 0.3s ease-in-out !important;
        }

        .custom-button:hover {
            background-color: #5b8ac4 !important;
        }

        .output-section {
            background-color: #ffffff !important;
            padding: 20px !important;
            border-radius: 12px !important;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05) !important;
        }

        .output-section h3 {
            color: #2b3a55 !important;
            text-align: center !important;
        }

        .result-table th {
            background-color: #2b3a55 !important; /* Navy gelap */
            color: #ffffff !important;            /* Teks putih cerah */
            font-weight: 600 !important;
            padding: 12px !important;
            text-align: left !important;
            border-bottom: 2px solid #cfd8dc !important;
        }

        .result-table td {
            background-color: #fdfdfd !important; /* Background terang */
            color: #212121 !important;            /* Teks gelap */
            padding: 12px !important;
            border-bottom: 1px solid #e0e0e0 !important;
        }

        .main-container {
            gap: 20px !important;
        }

        footer {
            display: none !important;
        }

    </style>
    """)


    # Event handler
    process_button.click(
        fn=process_titles_and_time,
        inputs=input_titles,
        outputs=[output_table, processing_time_text],
        )

## MAIN

In [ ]:
# Launch Gradio
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://66ddf5ac3fd4906c07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://66ddf5ac3fd4906c07.gradio.live
